In [289]:
import pandas as pd 
import numpy as np
import matplotlib as plt
import nltk 
import gzip
from scipy.sparse import csr_matrix


In [290]:
from sklearn.metrics.pairwise import cosine_similarity

In [291]:
df_ratings1 = pd.read_csv("ratings_Patio_Lawn_and_Garden.csv")
print(df_ratings1.shape)
df_ratings = df_ratings1[:10000]
print(df_ratings.shape)


(993489, 4)
(10000, 4)


In [292]:
# To read the reviews data 

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df_reviews = getDF("reviews_Patio_Lawn_and_Garden_5.json.gz")


In [293]:
df_ratings.head()

,A2VNYWOPJ13AFP,0981850006,5.0,1259798400
0,A20DWVV8HML3AW,0981850006,5.0,1371081600
1,A3RVP3YBYYOPRH,0981850006,5.0,1257984000
2,A28XY55TP3Q90O,0981850006,5.0,1314144000
3,A3VZW1BGUQO0V3,0981850006,5.0,1308268800
4,A2R9T5D7UVQZB0,0981850006,5.0,1253577600


In [294]:
df_ratings = df_ratings.rename(columns={"A2VNYWOPJ13AFP":"UserID","0981850006":"ObjectID","5.0":"Rating","1259798400":"UnixTime"})
df_ratings.head()

,UserID,ObjectID,Rating,UnixTime
0,A20DWVV8HML3AW,0981850006,5.0,1371081600
1,A3RVP3YBYYOPRH,0981850006,5.0,1257984000
2,A28XY55TP3Q90O,0981850006,5.0,1314144000
3,A3VZW1BGUQO0V3,0981850006,5.0,1308268800
4,A2R9T5D7UVQZB0,0981850006,5.0,1253577600


In [295]:
df_ratings["ObjectID"] = df_ratings["ObjectID"].astype("string")


In [296]:
df_ratings["UserID"] = df_ratings["UserID"].astype("string")

In [297]:
df_ratings["Rating"] = df_ratings["Rating"]/5  

### Only needed data for recommendation system is User ID and objectID

Step 1: Create a sparse matrix out of the features you require (user rating, correlation, usw.)

Step 2: Can you Nearest Neighbours to get the distances of objects in this sparse matrix w.r.t to any reference provided. 

In [298]:
df_ratings.head()

,UserID,ObjectID,Rating,UnixTime
0,A20DWVV8HML3AW,0981850006,1.0,1371081600
1,A3RVP3YBYYOPRH,0981850006,1.0,1257984000
2,A28XY55TP3Q90O,0981850006,1.0,1314144000
3,A3VZW1BGUQO0V3,0981850006,1.0,1308268800
4,A2R9T5D7UVQZB0,0981850006,1.0,1253577600


In [299]:
df_ratings["UserID"][0]

'A20DWVV8HML3AW'

In [300]:
df_ratings = df_ratings.drop("UnixTime",axis=1)


In [301]:
df_ratings.head()
print(df_ratings.shape)
df_ratings = df_ratings.drop_duplicates()
print(df_ratings.shape)

(10000, 3)
(10000, 3)


In [302]:
df_pivot = (df_ratings.pivot(index="UserID",columns="ObjectID",values="Rating").fillna(0))
df_pivot = df_pivot.T

df_pivot.head()


UserID,A0149220O8WD7KG3O5XR,A01678822I42RON4V4A84,A0326148384CSDVOOAIE2,A04122423ACFU9RNT481Y,A049365138OQHQR8907BX,A054583617DLR4GNAMW2R,A06720182VHL0J3VNHC1T,A08605632D162LVBW1IGS,A09874691WM5M8FOTE96D,A1001MAAQ3DQPE,...,AZV26LP92E6WU,AZV2T7ZAXA28V,AZV2U6GU5QA6C,AZVF084DOTOKU,AZVIQBBQQANY9,AZWMBFVSZ66G2,AZWZAT6H68T49,AZX2RDN9YXZAE,AZX4CLGYSU1HA,AZXMYR151RLL
ObjectID,,,,,,,,,,,,,,,,,,,,,
0981850006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144072007X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1554701503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1579822932,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1754164498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [303]:
sparse_matrix = csr_matrix(df_pivot.values)



Collaborative Filtering works by getting a similarity score,

Here we use cosine similarity

In [304]:
from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(algorithm="brute", metric="euclidean")
nn.fit(sparse_matrix)

NearestNeighbors(algorithm='brute', metric='euclidean')

### Getting Recommendations

In [305]:
random_user = np.random.choice(df_pivot.shape[0])

In [306]:
distance,indices = nn.kneighbors(df_pivot.iloc[random_user,:].values.reshape(1,-1),n_neighbors=100)

### Printing Output

In [307]:
for i in range(0,len(distance.flatten())):
    if i == 0:
        print("5 recommendations for {} ObjectID are: \n",format(df_pivot.index[random_user]))
    else:
        print("{0}:{1}, with a rating of {2}".format(i,df_pivot.index[indices.flatten()[i]],distance.flatten()[i]))

5 recommendations for {} ObjectID are: 
 B00002N6AG
1:B00002N8NF, with a rating of 1.6970562748477143
2:B00002N8JP, with a rating of 1.6970562748477143
3:B00002N7Q2, with a rating of 1.6970562748477143
4:B00002N6EB, with a rating of 1.6970562748477143
5:B00002N8JE, with a rating of 1.6970562748477143
6:9822221274, with a rating of 1.6970562748477143
7:B00002N68E, with a rating of 1.6970562748477143
8:B00002N68K, with a rating of 1.6970562748477143
9:B00002N8L1, with a rating of 1.6970562748477143
10:B00002N8NS, with a rating of 1.6970562748477143
11:B00002NCBA, with a rating of 1.6970562748477143
12:B00004R9PH, with a rating of 1.6970562748477143
13:B00002NCGM, with a rating of 1.6970562748477143
14:B00002N926, with a rating of 1.6970562748477143
15:B00004R9VT, with a rating of 1.6970562748477143
16:B00002ND6F, with a rating of 1.6970562748477143
17:B00002N65P, with a rating of 1.6970562748477143
18:B00002N65E, with a rating of 1.6970562748477143
19:B00002N67C, with a rating of 1.69705